## 여러 모델 사용하기
* IBM sample datasets
https://www.kaggle.com/blastchar/telco-customer-churn

* Demographic info: 
    * Gender, SeniorCitizen, Partner, Dependents
* Services subscribed: 
    * PhoneService, MultipleLine, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
* Customer account info:
    * CustomerID, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges, Tenure

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')

## 데이터 로드하기

In [7]:
df = pd.read_csv("data/telco_feature.csv")
df.shape

(7043, 50)

In [8]:
df = df.set_index("customerID")

In [9]:
df.head()

,Churn_bool,Dependents_bool,PhoneService_bool,gender_bool,Partner_bool,PaperlessBilling_bool,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,PaymentMethod_Bank transfer (automatic),...,tenure_group_0-20,tenure_group_20-40,tenure_group_40-60,tenure_group_60plus,InternetService_DSL,InternetService_Fiber optic,InternetService_No,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes
customerID,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,False,False,False,False,True,True,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0
5575-GNVDE,False,False,True,True,False,False,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3668-QPYBK,True,False,True,True,False,True,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0
7795-CFOCW,False,False,False,True,False,False,0,0,1,1,...,0,0,1,0,1,0,0,1,0,0
9237-HQITU,True,False,True,False,False,True,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype
---  ------                                   --------------  -----
 0   Churn_bool                               7043 non-null   bool 
 1   Dependents_bool                          7043 non-null   bool 
 2   PhoneService_bool                        7043 non-null   bool 
 3   gender_bool                              7043 non-null   bool 
 4   Partner_bool                             7043 non-null   bool 
 5   PaperlessBilling_bool                    7043 non-null   bool 
 6   DeviceProtection_No                      7043 non-null   int64
 7   DeviceProtection_No internet service     7043 non-null   int64
 8   DeviceProtection_Yes                     7043 non-null   int64
 9   PaymentMethod_Bank transfer (automatic)  7043 non-null   int64
 10  PaymentMethod_Credit card (automatic)    7043 non-null   int64

## 전처리

In [ ]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [ ]:
# 결측치를 채워주는 방법도 있지만 일단 제거하도록 합니다.
df = df.dropna()

In [18]:
df["Churn_bool"].value_counts()

False    5174
True     1869
Name: Churn_bool, dtype: int64

## 학습, 예측 데이터셋 나누기
### 학습, 예측에 사용할 컬럼

In [ ]:
df.nunique()

### 정답값이자 예측해야 될 값

In [11]:
# label_name 이라는 변수에 예측할 컬럼의 이름을 담습니다.
label_name = 'Churn_bool'

### 문제(feature)와 답안(label)을 나누기

* X, y를 만들어 줍니다.
* X는 feature, 독립변수, 예) 시험의 문제
* y는 label, 종속변수, 예) 시험의 정답

In [12]:
# X, y를 만들어 줍니다.
X = df.drop(label_name,axis=1)
y = df[label_name]

### 학습, 예측 데이터셋 만들기
* X_train : 학습 세트 만들기, 행렬, 판다스의 데이터프레임, 2차원 리스트(배열) 구조,  예) 시험의 기출문제
* y_train : 정답 값을 만들기, 벡터, 판다스의 시리즈, 1차원 리스트(배열) 구조, 예) 기출문제의 정답
* X_test : 예측에 사용할 데이터세트를 만듭니다. 예) 실전 시험 문제
* y_test : 예측의 정답값 예) 실전 시험 문제의 정답

In [13]:
# train_test_split 으로 데이터셋을 나눕니다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4718, 48), (2325, 48), (4718,), (2325,))

In [15]:
X_train.head(3)

,Dependents_bool,PhoneService_bool,gender_bool,Partner_bool,PaperlessBilling_bool,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),...,tenure_group_0-20,tenure_group_20-40,tenure_group_40-60,tenure_group_60plus,InternetService_DSL,InternetService_Fiber optic,InternetService_No,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes
customerID,,,,,,,,,,,,,,,,,,,,,
2985-JUUBZ,True,True,True,True,True,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
5016-ETTFF,False,False,True,False,False,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,0
1998-VHJHK,False,True,False,False,False,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [16]:
X_test.head(3)

,Dependents_bool,PhoneService_bool,gender_bool,Partner_bool,PaperlessBilling_bool,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),...,tenure_group_0-20,tenure_group_20-40,tenure_group_40-60,tenure_group_60plus,InternetService_DSL,InternetService_Fiber optic,InternetService_No,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes
customerID,,,,,,,,,,,,,,,,,,,,,
1024-GUALD,False,False,False,True,True,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
0484-JPBRU,False,True,True,False,True,0,1,0,1,0,...,0,0,1,0,0,0,1,0,1,0
3620-EHIMZ,True,True,False,True,False,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [17]:
y_train.head(2)

customerID
2985-JUUBZ    False
5016-ETTFF     True
Name: Churn_bool, dtype: bool

## 머신러닝 모델로 예측하기

In [19]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

In [20]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, X_train, y_train, cv=5, verbose=2, n_jobs=-1,)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.5s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s finished


In [22]:
(y_train == y_pred).mean()

0.7206443408223824

## 그리드 서치

In [27]:
max_depth_list = np.arange(3,15)
max_depth_list

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [30]:
max_features_list = np.arange(0.1, 1.1, step =0.1)
max_features_list

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [32]:
result_list = []
for max_depth in max_depth_list:
    for max_features in max_features_list:
        
        result= []
        model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
        y_pred = cross_val_predict(model, X_train, y_train, cv=5, verbose=2, n_jobs=-1,)
        accuracy = (y_train == y_pred).mean()
        result.append(max_depth)
        result.append(max_features)
        result.append(accuracy)
        result_list.append(result)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent worker

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining: 

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining: 

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining: 

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining: 

## 랜덤서치

In [44]:
max_depth = np.random.randint(1,10)
max_features = np.random.uniform(0.1,1.0)
max_depth, max_features

(3, 0.9105073397198032)

In [48]:
for i in range(10):
    max_depth = np.random.randint(1,10)
    max_features = np.random.uniform(0.1,1.0)
    
    result= []
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    y_pred = cross_val_predict(model, X_train, y_train, cv=5, verbose=2, n_jobs=-1,)
    accuracy = (y_train == y_pred).mean()
    result.append(max_depth)
    result.append(max_features)
    result.append(accuracy)
    result_list.append(result)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.3s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent worker

In [23]:
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [24]:
y_test_pred = model.predict(X_test)

(y_test_pred == y_test).mean()

0.730752688172043

## 모델 평가하기

In [ ]:
# 피처의 중요도를 추출하기


In [ ]:
# 피처의 중요도 시각화 하기


### 점수 측정하기